In [1]:
import pandas as pd
import geemap
import ee

In [2]:
Map = geemap.Map(center=(29, 85))
Map

Map(center=[29, 85], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=Fa…

In [7]:
roi_f = geemap.shp_to_ee(r'E:/Data/temp/rec/roi.shp')
Map.addLayer(roi_f, {}, 'roi')

In [8]:
roi = roi_f.geometry()

In [37]:
images = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")\
        .filterDate('1990', '1991')\
        .filterBounds(roi).sort("CLOUD_COVER", False)

In [ ]:
def rmCloudByQA(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
               .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

In [41]:
def bitwiseExtract(value, fromBit, toBit=None):
    if toBit == None:
        toBit = fromBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

def cloudfree_landsat(img):
    qa = img.select('pixel_qa')
    cloudState = bitwiseExtract(qa, 5)
    cloudShadowState = bitwiseExtract(qa, 3)
    mask = cloudState.eq(0).And(cloudShadowState.eq(0))
    return img.updateMask(mask)

In [42]:
cf_image = images.map(cloudfree_landsat)

In [49]:
image = cf_image.median().clip(roi)

In [50]:
# img = images.median().select(['B4', 'B3', 'B2']).clip(roi)
Map.addLayer(image, {'bands':['B3', 'B2', 'B1'], 'max': 3000, 'min':0}, 'img_1')

In [51]:
geemap.ee_export_image_to_drive(image.select(['B3', 'B2', 'B1']), description='1990RGB', 
                                scale=30, folder='Landsat')

Exporting 1990RGB ...


In [ ]:
NDVI = image.normalizedDifference(['B5', 'B4'])
img = image.select(['B4', 'B3', 'B2'])

In [ ]:
geemap.ee_export_image_to_drive(NDVI, description='NDVI', 
                                scale=30, folder='Landsat8_xizhang')

In [ ]:
a = """
function rmCloudByQA(image) { 
  var qa = image.select('QA60'); 
  var cloudBitMask = 1 << 10; 
  var cirrusBitMask = 1 << 11; 
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0) 
               .and(qa.bitwiseAnd(cirrusBitMask).eq(0)); 
  return image.updateMask(mask); 
} 
"""

In [ ]:
geemap.js_snippet_to_py(a)

In [53]:
ee.List.sequence(0, 2).getInfo()

[0, 1, 2]